In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL structure
BASE_URL = "https://www.rootsrwanda.rw"
CATEGORY_ID = "MQ=="  # Mobiles & Tablets

# Brands with their base64 subcategory IDs
brands = {
    "Samsung": "Mg==",   # 1
    "Nokia": "Mw==",     # 2
    "Huawei": "NA==",    # 3
    "Oppo": "NQ==",      # 4
    "Apple": "MQ=="      # 5 ← newly added
}

# List to store all scraped products
all_products = []

# Loop through each brand
for brand, subcategory_id in brands.items():
    # print(f"Scraping {brand} products...")
    
    url = f"{BASE_URL}/product-list/{CATEGORY_ID}/{subcategory_id}/Mobiles-&-Tablets/{brand}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to load {brand} page. Status code: {response.status_code}")
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    product_blocks = soup.find_all('div', class_='right-block')
    
    for block in product_blocks:
        name_tag = block.find('h4').find('a')
        if not name_tag:
            continue

        product_name = name_tag.get_text(strip=True)
        product_url = name_tag['href']
        if not product_url.startswith("http"):
            product_url = BASE_URL + product_url

        price_tag = block.find('span', class_='price-new')
        product_price = price_tag.get_text(strip=True).replace("Rwf", "").strip() if price_tag else "Not specified"

        all_products.append({
            "Brand": brand,
            "Product Name": product_name,
            "Product Price": product_price,
            "Product URL": product_url
        })

    time.sleep(1)

# Save results to CSV
if all_products:
    df = pd.DataFrame(all_products)
    print(df.head())
    df.to_csv("Mobiles_&_Tablets.csv", index=False)
    print(f"\n Scraped {len(df)} products across {len(brands)} brands.")
   
else:
    print(" No products found.")


     Brand                                       Product Name Product Price  \
0  Samsung  Samsung Galaxy S25 Plus-S25+ Mobile Phone, Kig...     1,480,000   
1  Samsung  Samsung Galaxy S25 Mobile Phone, Kigali Rwanda...     1,190,000   
2  Samsung  Samsung A56 256GB, 8GB RAM Mobile phone, New m...       610,000   
3  Samsung  Samsung A56 128GB, 6GB RAM Mobile phone, New m...       570,000   
4  Samsung  Samsung A36 256GB, 6GB RAM Mobile phone, New m...       510,000   

                                         Product URL  
0  https://www.rootsrwanda.rw/product-detail/MzAz...  
1  https://www.rootsrwanda.rw/product-detail/MzAy...  
2  https://www.rootsrwanda.rw/product-detail/MzAy...  
3  https://www.rootsrwanda.rw/product-detail/MzAy...  
4  https://www.rootsrwanda.rw/product-detail/MzAy...  

 Scraped 209 products across 5 brands.
